<a href="https://colab.research.google.com/github/AmandaJMendes/muNet/blob/main/muNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data

In [1]:
import pandas as pd
data = pd.read_csv("one_hot_cancer.csv")
data.drop("Unnamed: 0", axis = 1, inplace = True)
n_20 = int(len(data)*0.2)
test = data.sample(n = n_20)
train_val = data.drop(test.index)
val = train_val.sample(n = n_20)
train = train_val.drop(val.index)
x_test = test.iloc[:, :-2]
y_test = test.iloc[:, -2]
x_val = val.iloc[:, :-2]
y_val = val.iloc[:, -2]
x_train = train.iloc[:, :-2]
y_train = train.iloc[:, -2]

In [2]:
data2 = pd.read_csv("Breast_Cancer.csv")
data2.rename(columns={'T Stage ': 'T Stage'}, inplace=True)
ndata = data2.copy()

for i in [3, 4, 5, 8, 10, 11, 15]:
  ndata.iloc[:, i] = ndata.iloc[:, i].astype("category").cat.codes
ndata.iloc[:, 6] = ndata.iloc[:, 6].astype("category").cat.set_categories(['Undifferentiated',
                                                                           'Poorly differentiated',
                                                                           'Moderately differentiated',
                                                                           'Well differentiated'],
                                                                          ordered = True).cat.codes

ndata.iloc[:, 7] = ndata.iloc[:, 7].astype("category").cat.set_categories(['1', '2', '3', ' anaplastic; Grade IV'],
                                                                          ordered = True).cat.codes

ndata = pd.get_dummies(ndata, columns = ["Race", "Marital Status"])
ndata = ndata.astype("int64")  
n_20_2 = int(len(ndata)*0.2)
test2 = ndata.sample(n = n_20)
train_val2 = ndata.drop(test2.index)
val2 = train_val2.sample(n = n_20_2)
train2 = train_val2.drop(val2.index)
x_test2 = test2.iloc[:, :-2]
y_test2 = test2.iloc[:, -2]
x_val2 = val2.iloc[:, :-2]
y_val2 = val2.iloc[:, -2]
x_train2 = train2.iloc[:, :-2]
y_train2 = train2.iloc[:, -2]

# Create root model

In [127]:
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.activations import relu, sigmoid
from tensorflow.keras import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.models import load_model, clone_model
import numpy as np

In [128]:
model = Sequential([
        Flatten(input_shape=(39,)),
        Dense(32, activation=relu),
        Dense(32, activation=relu),
	      Dense(16, activation=relu),
        Dense(8, activation=relu),
        Dense(1, activation=sigmoid),])

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['acc'])
model.save("root_model.h5")

# MuNet
Attempt to clone optimizer state
(I guess it works, but I'm not happy with it)

In [207]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

class muNet:
  def __init__(self):
    
    self.tasks = [{"x_train": x_train, "y_train": y_train,
                   "x_val": x_val, "y_val": y_val},
                  {"x_train": x_train2, "y_train": y_train2,
                   "x_val": x_val2, "y_val": y_val2}]

    self.n = len(self.tasks)
    self.root_model = load_model("/content/root_model.h5")
    
    cols1 = pd.MultiIndex.from_tuples([("Model", ""),
                                       ("Learning Rate", ""),
                                       ("Batch Size", "")])
    cols2 = pd.MultiIndex.from_product([['Selected','Score'],
                                        [f'T{i}' for i in range(self.n)]])
    data1 = [[self.root_model, 0.01, 128]]
    data2 = [[0 for i in range(self.n)]+[-1.0 for i in range(self.n)]]

    self.models = pd.concat([pd.DataFrame(data1, columns = cols1),
                             pd.DataFrame(data2, columns = cols2)], axis=1)
    
    self.layers = pd.DataFrame([[l, l.name]+4*[None]+[[]] for l in self.root_model.layers],
                               columns = ["Layer", "Name", "Kernel_m", "Bias_m",
                                          "Kernel_v", "Bias_v", "Models"])
    self.layer_name = int(self.layers.tail(1).Name.values[0].split("_")[-1])
    self.active_pop = []
    self.active_task = None
    self.mutation_options = {"Learning Rate": [0.0001, 0.0002, 0.0005,
                                               0.001, 0.002, 0.005,
                                               0.01, 0.02, 0.05, 0.1],
                             "Batch Size": [1, 2, 4, 8, 16, 32, 64, 128, 256]}

  def sample_parent(self):
    active = self.models.loc[self.active_pop, :]
    sorted_active = list(active.sort_values(("Score", f"T{self.active_task}")).index)
    others = list(self.models.drop(self.active_pop).index)
    np.random.shuffle(others)
    options = sorted_active + others

    parent = None
    for model_idx in options:
      selections = self.models.loc[model_idx, ("Selected", f"T{self.active_task}")]
      if 0.5**selections > np.random.uniform():
        parent = model_idx
        break
    if not parent:
      parent = np.random.choice(options)

    self.models.at[parent, ("Selected", f"T{self.active_task}")] += 1
    return parent

  def sample_mutations(self, parent):
    new_hyper_params = {}

    new_hyper_params["Clone"] = []
    for layer in range(len(self.models.loc[parent, ("Model", "")].layers)):
      if np.random.uniform() > 0.5:
        print("clone")
        new_hyper_params["Clone"].append(layer)

    for key, options in self.mutation_options.items():
      current_hp = self.models.loc[parent, (key, "")] 
      change = 1 if np.random.uniform() > 0.5 else 0
      print(change)
      if change:
        current_hyperparam_idx = options.index(current_hp)
        if current_hyperparam_idx == 0:
          new_hyperparam_idx = current_hyperparam_idx + 1
        elif current_hyperparam_idx == len(options)-1:
          new_hyperparam_idx = current_hyperparam_idx - 1
        else:
          new_hyperparam_idx = current_hyperparam_idx + 1 if np.random.uniform() > 0.5 else current_hyperparam_idx - 1

        new_hyper_params[key] = options[new_hyperparam_idx]
      else:
        new_hyper_params[key] = current_hp
    return new_hyper_params

  def clone_layer(self, layer, change = None, in_shape = None, clone_w = False):
    print(layer.name, change, in_shape, clone_w)
    config = layer.get_config()
    for param, value in change.items():
      config[param] = value
    clone = type(layer).from_config(config)
    clone.build(in_shape if in_shape else layer.input_shape)
    if clone_w:
      clone.set_weights(layer.get_weights())
    clone.trainable = True    
    return clone

  def append_to_df(self, dataframe, rows):
    new_df = pd.DataFrame(rows, columns = dataframe.columns)
    return pd.concat([dataframe, new_df]).reset_index(drop=True)

  def create_name(self, name):
    self.layer_name+=1
    return ''.join([i for i in name if not i.isdigit()])+str(self.layer_name)

  def create_child(self, parent, mutations):
    parent_model = self.models.loc[parent, ("Model", "")]
    task_in_shape = self.tasks[self.active_task]["x_train"].shape
    task_out_shape = (None, 1)

    to_clone = mutations["Clone"]
    if 0 in to_clone:
      to_clone.remove(0)
    child_layers = []
    for layer_idx, layer in enumerate(parent_model.layers):
      print("layer name: ", self.layer_name)
      print(task_in_shape, layer.input_shape)
      if layer_idx == 0 and task_in_shape[1] != layer.input_shape[1]:
        clone = self.clone_layer(layer,
                                 change = {"name": self.create_name(layer.name)},
                                 in_shape = (task_in_shape[-1], ))
        child_layers.append(clone)
        self.layers = self.append_to_df(self.layers, [[clone, clone.name]+4*[None]+[[]]])

      elif layer_idx == 1 and task_in_shape[-1] != layer.input_shape[1]:
        clone = self.clone_layer(layer,
                                 change = {"name": self.create_name(layer.name)},
                                 in_shape = (None, task_in_shape[-1]))
        child_layers.append(clone)
        self.layers = self.append_to_df(self.layers, [[clone, clone.name]+4*[None]+[[]]])

      elif layer_idx == len(parent_model.layers)-1:
        print(layer.output.shape, task_out_shape)
        if layer.output.shape == task_out_shape:
          clone = self.clone_layer(layer,
                                   change = {"name": self.create_name(layer.name)},
                                   clone_w = True)
        else:
          clone = self.clone_layer(layer,
                                   change = {"units": task_out_shape[-1],
                                             "name": self.create_name(layer.name)})
        child_layers.append(clone)
        self.layers = self.append_to_df(self.layers, [[clone, clone.name]+4*[None]+[[]]])

      elif layer_idx in mutations["Clone"]:#or parent_model == self.root_model:
        clone = self.clone_layer(layer, change = {"name": self.create_name(layer.name)}, clone_w = True)
        child_layers.append(clone)
        self.layers = self.append_to_df(self.layers, [[clone, clone.name]+4*[None]+[[]]])

      else:
        layer.trainable = False
        child_layers.append(layer)
   
    child_model = Sequential(child_layers)
    child_model.compile(optimizer=optimizers.Adam(learning_rate=mutations["Learning Rate"]),
                        loss='binary_crossentropy',
                        metrics=['acc'])
    return child_model
    """
    grad_vars = child_model.trainable_weights
    zero_grads = [tf.zeros_like(w) for w in grad_vars]
    child_model.optimizer.apply_gradients(zip(zero_grads, grad_vars))    
    ms = []
    vs = []
    for layer in [l for l in child_model.layers if l.trainable]:
      layer_df = self.layers.loc[self.layers.Layer == layer]
      if layer_df.Kernel_m.values[0]:
        ms.append(layer_df.Kernel_m.values[0])
        ms.append(layer_df.Bias_m.values[0])
        vs.append(layer_df.Kernel_v.values[0])
        vs.append(layer_df.Bias_v.values[0])
      else:
        if len(layer.get_weights()):
          ms.append(tf.zeros_like(layer.get_weights()[0]))
          ms.append(tf.zeros_like(layer.get_weights()[1]))
          vs.append(tf.zeros_like(layer.get_weights()[0]))
          vs.append(tf.zeros_like(layer.get_weights()[1]))
    print([i.shape for i in child_model.optimizer.get_weights()])
    print([i.shape for i in ms])
    print([i.shape for i in vs])
    child_model.optimizer.set_weights([tf.Variable(0.0)]+ms+vs)
    return child_model
    """
  def train_child(self, child, bs):
    child.fit(self.tasks[self.active_task]["x_train"],
              self.tasks[self.active_task]["y_train"],
              epochs = 1,
              batch_size = bs)
    return child

  def score(self, model):
    loss, acc = model.evaluate(self.tasks[self.active_task]["x_val"],
                               self.tasks[self.active_task]["y_val"])
    return acc
  
  def evaluate(self, row, x_test, y_test):
    try:
      loss, acc = self.models.iloc[row, 0].evaluate(x_test, y_test)
    except RuntimeError:
      self.models.iloc[row,0].compile(optimizer=optimizers.Adam(),
                          loss='binary_crossentropy',
                          metrics=['acc'])
      loss, acc = self.models.iloc[row,0].evaluate(x_test, y_test)

    return self.models.iloc[[row]], acc, loss
  
  def train_system(self, generations, children):
    #Treinar mais de uma vez
    for cycle in range(4):
      for task in range(2):
        print("Task: ", task)
        self.active_task = task
        self.active_pop = list(self.models[self.models[("Score", f"T{task}")] >= 0].index)
        print("Active pop: ", self.active_pop)

        for gen in range(generations):
          for child in range(children):
            #Tentar manter apenas a melhor child da geração e não deixar que modelos da mesma geração sejam pais e filhos
            self.counter+=1
            parent = self.sample_parent()
            print("Parent: ", parent)
            mutations = self.sample_mutations(parent)
            print("Mutations: ", mutations)
            child = self.create_child(parent, mutations)
            print("Child layers: ", [i.trainable for i in child.layers])
            
            retained_child = None
            best_score = self.models.loc[parent, ("Score", f"T{task}")]
            for i in range(4):
              print("Best score: ", best_score)
              child = self.train_child(child, mutations["Batch Size"])
              print("Child: ", child)
              child_score = self.score(child)
              print("Child score: ", child_score)
              if child_score > best_score:
                best_score = child_score

                retained_child = clone_model(child)
                retained_child.set_weights(child.get_weights()) 
                symbolic_weights = getattr(child.optimizer, 'weights') 
                retained_opt = K.batch_get_value(symbolic_weights) 
                retained_child.compile(optimizer=optimizers.Adam(),
                                        loss='binary_crossentropy',
                                        metrics=['acc'])
                grad_vars = child.trainable_weights
                zero_grads = [tf.zeros_like(w) for w in grad_vars]
                retained_child.optimizer.apply_gradients(zip(zero_grads, grad_vars))
                retained_child.optimizer.set_weights(retained_opt)
                
              
            if retained_child:
              k = 1
              new_rows = []
              tls = [l for l in retained_child.layers if l.trainable]
              if type(tls[0]) == Flatten:
                tls = tls[1:]
              for layer in tls:
                if not type(layer) == Flatten:
                  layer_row = self.layers[self.layers.Layer == layer]
                  self.layers.drop(layer_row.index)
                  print([a.shape for a in retained_child.optimizer.get_weights()])
                  print(k)
                  new_rows.append([layer,
                                  retained_child.optimizer.get_weights()[k],
                                  retained_child.optimizer.get_weights()[k+1],
                                  retained_child.optimizer.get_weights()[k+len(tls)*2],
                                  retained_child.optimizer.get_weights()[k+len(tls)*2+1],
                                  []])
                  k += 2
              self.layers = pd.concat([self.layers,
                                       pd.DataFrame(new_rows, columns = self.layers.columns)])
              self.models = pd.concat([self.models,
                                      pd.DataFrame([[retained_child] + \
                                                    [mutations["Learning Rate"], mutations["Batch Size"]] + \
                                                    [0 for i in range(self.n)]+[-1.0 for i in range(self.n)]],
                                                    columns = self.models.columns)]).reset_index(drop=True)                                     
              self.models.iat[-1, self.models.columns.get_loc(("Score", f"T{task}"))] = best_score
              self.active_pop.append(self.models.index[-1])

              
        if self.active_pop:
          print("Active pop end: ", self.active_pop)
          active_pop_scores = self.models.loc[self.active_pop, ("Score", f"T{task}")]
          best_row = self.models.loc[[active_pop_scores.idxmax()]]

          not_trained = self.models.drop(active_pop_scores.index)

          self.models = pd.concat([not_trained, best_row]).reset_index(drop=True)
          print("Models end: ", self.models)
          #Ideia: não manter só o melhor. Criar métrica que considera se está acima de um threshold e já foi usado como pai tantas vezes
        print("Models at the end of task:\n", self.models)

# MuNet (newest version)

In [242]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

class muNet:
  def __init__(self):
    
    self.tasks = [{"x_train": x_train, "y_train": y_train,
                   "x_val": x_val, "y_val": y_val},
                  {"x_train": x_train2, "y_train": y_train2,
                   "x_val": x_val2, "y_val": y_val2}]

    self.n = len(self.tasks)
    self.root_model = load_model("/content/root_model.h5")
    
    cols1 = pd.MultiIndex.from_tuples([("Model", ""),
                                       ("Learning Rate", ""),
                                       ("Batch Size", "")])
    cols2 = pd.MultiIndex.from_product([['Selected','Score'],
                                        [f'T{i}' for i in range(self.n)]])
    data1 = [[self.root_model, 0.01, 128]]
    data2 = [[0 for i in range(self.n)]+[-1.0 for i in range(self.n)]]

    self.models = pd.concat([pd.DataFrame(data1, columns = cols1),
                             pd.DataFrame(data2, columns = cols2)], axis=1)
    self.layers = pd.DataFrame([[l, l.name]+4*[None]+[[]] for l in self.root_model.layers],
                               columns = ["Layer", "Name", "Kernel_m", "Bias_m",
                                          "Kernel_v", "Bias_v", "Models"])
    self.layer_name = int(self.layers.tail(1).Name.values[0].split("_")[-1])
    self.active_pop = []
    self.active_task = None
    self.mutation_options = {"Learning Rate": [0.0001, 0.0002, 0.0005,
                                               0.001, 0.002, 0.005,
                                               0.01, 0.02, 0.05, 0.1],
                             "Batch Size": [1, 2, 4, 8, 16, 32, 64, 128, 256]}

  def sample_parent(self):
    active = self.models.loc[self.active_pop, :]
    sorted_active = list(active.sort_values(("Score", f"T{self.active_task}")).index)
    others = list(self.models.drop(self.active_pop).index)
    np.random.shuffle(others)
    options = sorted_active + others

    parent = None
    for model_idx in options:
      selections = self.models.loc[model_idx, ("Selected", f"T{self.active_task}")]
      if 0.5**selections > np.random.uniform():
        parent = model_idx
        break
    if not parent:
      parent = np.random.choice(options)

    self.models.at[parent, ("Selected", f"T{self.active_task}")] += 1
    return parent

  def sample_mutations(self, parent):
    new_hyper_params = {}

    new_hyper_params["Clone"] = []
    for layer in range(1, len(self.models.loc[parent, ("Model", "")].layers)):
      if np.random.uniform() > 0.5:
        new_hyper_params["Clone"].append(layer)

    for key, options in self.mutation_options.items():
      current_hp = self.models.loc[parent, (key, "")] 
      change = 1 if np.random.uniform() > 0.5 else 0
      if change:
        current_hyperparam_idx = options.index(current_hp)
        if current_hyperparam_idx == 0:
          new_hyperparam_idx = current_hyperparam_idx + 1
        elif current_hyperparam_idx == len(options)-1:
          new_hyperparam_idx = current_hyperparam_idx - 1
        else:
          new_hyperparam_idx = current_hyperparam_idx + 1 if np.random.uniform() > 0.5 else current_hyperparam_idx - 1

        new_hyper_params[key] = options[new_hyperparam_idx]
      else:
        new_hyper_params[key] = current_hp
    return new_hyper_params

  def clone_layer(self, layer, change = None, in_shape = None, clone_w = False):

    config = layer.get_config()
    for param, value in change.items():
      config[param] = value
    clone = type(layer).from_config(config)
    clone.build(in_shape if in_shape else layer.input_shape)
    if clone_w:
      clone.set_weights(layer.get_weights())
    clone.trainable = True  
    if type(layer)==Flatten:
      print(layer.get_config())
      print(config)  
      print(in_shape)
    return clone

  def append_to_df(self, dataframe, rows):
    new_df = pd.DataFrame(rows, columns = dataframe.columns)
    return pd.concat([dataframe, new_df]).reset_index(drop=True)

  def create_name(self, name):
    self.layer_name+=1
    return ''.join([i for i in name if not i.isdigit()])+str(self.layer_name)

  def create_child(self, parent, mutations):
    parent_model = self.models.loc[parent, ("Model", "")]
    task_in_shape = self.tasks[self.active_task]["x_train"].shape
    task_out_shape = (None, 1)
    to_clone = mutations["Clone"]
    child_layers = []

    for layer_idx, layer in enumerate(parent_model.layers):
      if layer_idx == 0 and task_in_shape[1] != layer.input_shape[1]:
        clone = self.clone_layer(layer,
                                 change = {"name": self.create_name(layer.name),
                                           "batch_input_shape": (None, task_in_shape[-1])},
                                 in_shape = (task_in_shape[-1], ))
        child_layers.append(clone)
        self.layers = self.append_to_df(self.layers, [[clone, clone.name]+4*[None]+[[]]])

      elif layer_idx == 1 and task_in_shape[-1] != layer.input_shape[1]:
        clone = self.clone_layer(layer,
                                 change = {"name": self.create_name(layer.name)},
                                 in_shape = (None, task_in_shape[-1]))
        child_layers.append(clone)
        self.layers = self.append_to_df(self.layers, [[clone, clone.name]+4*[None]+[[]]])

      elif layer_idx == len(parent_model.layers)-1:
        if layer.output.shape == task_out_shape:
          clone = self.clone_layer(layer,
                                   change = {"name": self.create_name(layer.name)},
                                   clone_w = True)
        else:
          clone = self.clone_layer(layer,
                                   change = {"units": task_out_shape[-1],
                                             "name": self.create_name(layer.name)})
        child_layers.append(clone)
        self.layers = self.append_to_df(self.layers, [[clone, clone.name]+4*[None]+[[]]])

      elif layer_idx in mutations["Clone"]:
        clone = self.clone_layer(layer, change = {"name": self.create_name(layer.name)}, clone_w = True)
        child_layers.append(clone)
        self.layers = self.append_to_df(self.layers, [[clone, clone.name]+4*[None]+[[]]])

      else:
        layer.trainable = False
        child_layers.append(layer)
   
    child_model = Sequential(child_layers)
    child_model.compile(optimizer=optimizers.Adam(learning_rate=mutations["Learning Rate"]),
                        loss='binary_crossentropy',
                        metrics=['acc'])
    return child_model

  def train_child(self, child, bs):
    child.fit(self.tasks[self.active_task]["x_train"],
              self.tasks[self.active_task]["y_train"],
              epochs = 1,
              batch_size = bs)
    return child

  def score(self, model):
    loss, acc = model.evaluate(self.tasks[self.active_task]["x_val"],
                               self.tasks[self.active_task]["y_val"])
    return acc
  
  def train_system(self, generations, children):
    for cycle in range(4):
      print("Cycle ", cycle)
      for task in range(2):
        print("Task ", task)
        self.active_task = task
        self.active_pop = list(self.models[self.models[("Score", f"T{task}")] >= 0].index)    
        for gen in range(generations):
          for child in range(children):
            print("Active pop: ", self.active_pop)
            parent = self.sample_parent()
            print("Parent: ", [l.name for l in self.models.loc[parent, ("Model", "")].layers])
            mutations = self.sample_mutations(parent)
            child = self.create_child(parent, mutations)
            print("Child: ", [l.name for l in child.layers])
            retained_child = None
            best_score = self.models.loc[parent, ("Score", f"T{task}")]
            for i in range(4):
              child = self.train_child(child, mutations["Batch Size"])
              child_score = self.score(child)

              if child_score > best_score:
                best_score = child_score
                retained_child = clone_model(child)
                retained_child.set_weights(child.get_weights()) 
                retained_child.compile(optimizer=optimizers.Adam(),
                                        loss='binary_crossentropy',
                                        metrics=['acc'])
                
                symbolic_weights = getattr(child.optimizer, 'weights') 
                retained_opt = K.batch_get_value(symbolic_weights) 
                grad_vars = child.trainable_weights
                zero_grads = [tf.zeros_like(w) for w in grad_vars]
                retained_child.optimizer.apply_gradients(zip(zero_grads, grad_vars))
                retained_child.optimizer.set_weights(retained_opt)
                         
            if retained_child:
              self.models = pd.concat([self.models,
                                      pd.DataFrame([[retained_child] + \
                                                    [mutations["Learning Rate"], mutations["Batch Size"]] + \
                                                    [0 for i in range(self.n)]+[-1.0 for i in range(self.n)]],
                                                    columns = self.models.columns)]).reset_index(drop=True)                                     
              self.models.iat[-1, self.models.columns.get_loc(("Score", f"T{task}"))] = best_score
              self.active_pop.append(self.models.index[-1])
              
        if self.active_pop:
          active_pop_scores = self.models.loc[self.active_pop, ("Score", f"T{task}")]
          best_row = self.models.loc[[active_pop_scores.idxmax()]]
          not_trained = self.models.drop(active_pop_scores.index)
          self.models = pd.concat([not_trained, best_row]).reset_index(drop=True)
          print("Models (task end): ", self.models)


In [243]:
mtl = muNet()
mtl.train_system(2, 2)

Cycle  0
Task  0
Active pop:  []
Parent:  ['flatten_5', 'dense_25', 'dense_26', 'dense_27', 'dense_28', 'dense_29']
Child:  ['flatten_5', 'dense_25', 'dense_26', 'dense_27', 'dense_30', 'dense_31']
26/26 [==============================] - 0s 1ms/step - loss: 0.3197 - acc: 0.8806
Active pop:  [1]
Parent:  ['flatten_5', 'dense_25', 'dense_26', 'dense_27', 'dense_30', 'dense_31']
Child:  ['flatten_5', 'dense_32', 'dense_26', 'dense_27', 'dense_33', 'dense_34']
26/26 [==============================] - 0s 1ms/step - loss: 0.2978 - acc: 0.8905
Active pop:  [1, 2]
Parent:  ['flatten_5', 'dense_25', 'dense_26', 'dense_27', 'dense_30', 'dense_31']
Child:  ['flatten_5', 'dense_25', 'dense_26', 'dense_35', 'dense_30', 'dense_36']
26/26 [==============================] - 0s 1ms/step - loss: 0.3049 - acc: 0.8893
Active pop:  [1, 2, 3]
Parent:  ['flatten_5', 'dense_25', 'dense_26', 'dense_35', 'dense_30', 'dense_36']
Child:  ['flatten_5', 'dense_37', 'dense_38', 'dense_35', 'dense_39', 'dense_40']
2